In [ ]:
import pandas as pd 
import numpy  as np
import json

file_path = '/home/mantovani/Documents/IC/HMC_TE-s/Temp_Data/Dados_Dominios_Conservados/sequences.json'

df:dict = {'Name': []}

with open(file_path, mode='r') as file:

    dados = json.load(file)

    for i in dados['results']:

        df['Name'].append(i['crossReferences'][0]['name'].strip()[0:11])

        l_reading_frames, l_df = len(i['openReadingFrames']), (len(df)-1)//4

        print(i['openReadingFrames'],'\n', len(i['openReadingFrames']))

        for j in range(max(l_reading_frames, l_df)):

            reading_frames = i['openReadingFrames'][j]

            if j > l_df or l_df == 0:

                l = [np.nan for _ in range(len(df['Name'])-1)]#pois o nome é gerado antes

                df['Start_{}'.format(j)]             = l
                df['End_{}'.format(j)]               = l
                df['Strand_{}'.format(j)]            = l
                df['Protein_Sequence_{}'.format(j)]  = l

            elif j <= l_reading_frames:

                df['Start_{}'.format(j)].append(reading_frames['start'])
                df['End_{}'.format(j)].append(reading_frames['end'])
                df['Strand_{}'.format(j)].append(reading_frames['strand'])
                df['Protein_Sequence_{}'.format(j)].append(reading_frames['protein']['sequence'])

            else:

                df['Start_{}'.format(j)].append(np.nan)
                df['End_{}'.format(j)].append(np.nan)
                df['Strand_{}'.format(j)].append(np.nan)
                df['Protein_Sequence_{}'.format(j)].append(np.nan)

    file.close()

In [5]:
import pandas as pd 

meu_dict = {
    'chave1': ['subchave1', 'subvalor1', 'subchave2', 'subvalor2'],
    'chave2': ['valor2']
} 

meu_dict = pd.DataFrame(meu_dict)

meu_dict.head()

,chave1,chave2
0,subchave1,valor2
1,subvalor1,NaN
2,subchave2,NaN
3,subvalor2,NaN


In [16]:
print([np.nan for i in range(0)])

[]
